## Exercise 
創建一個RNN網路,能夠預測下一個數字


In [2]:
# %load simpleRNN.py
'''
 RNN model type: Many to one
 here, we use sequential model to create a single output tensor,
 For multi-output layers, use the functional API.
 https://keras.io/guides/functional_api/
'''

from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
import numpy as np

In [3]:


# Function to genate the input
# You can try with different input
def sequence_generator(typeof):
	# [0, 10, 20, ...]
	if typeof == 'plusten':
		sequence = [i for i in range(0, 1500, 10)]
	# [2, 4, 6, ...]
	elif typeof == 'double':
		sequence = [i*2 for i in range(1, 250)]
	# [1, 4, 9, 25, ...]
	elif typeof == 'power':
		sequence = [i**2 for i in range(1, 100)]
	return sequence


#  To separate the sequence into input and output ([X] [y])
# For example [1, 3, 5, 7, 9, 11] becomes
# [1, 3, 5] --> [7]
# [3, 5, 7] --> [9]
# [5, 7, 9] --> [11] 
def separate_sequence(sequence, seq_len):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + seq_len
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 




## Generate the input sequence

In [4]:
time_step = 4
n_features = 1

t = 'plusten'
sq = sequence_generator(t)
print (sq)

# split into samples
X, y = separate_sequence(sq,time_step)

# reshape the input into [batch, timesteps, features] to be readable to the network
print(X[:5])
print(y[:5])
X = X.reshape((X.shape[0], X.shape[1], n_features))
print('X.shape',X.shape)
print('y.shape',y.shape)

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470, 1480, 1490]
[[ 0 10 20 30]
 [10 20 30 40]
 [20 30 40 50]
 [30 40 50 60]
 [40 50 60 70]]
[40 50 60 70 80]
X.shape (146, 4, 1)
y.shape (146,)


## create model and train model

In [5]:
model = Sequential()
#model.add(Bidirectional(LSTM(64, activation='relu'), input_shape=(time_step, n_features)))
model.add(LSTM(128, activation='relu',input_shape=(time_step, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mae')
model.summary()
# fit model
model.fit(X, y, epochs=400, batch_size =32,verbose=0)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               66560     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


## Test

In [6]:
# Input a sequence and predict
x_input = array([39,49,59,69])
x_input = x_input.reshape((1, time_step, n_features))
yhat = model.predict(x_input, verbose=0)
print(np.round(yhat))

[[79.]]


In [7]:
print(X[:10])
yhat = model.predict(X[:10], verbose=0)
np.round(yhat)

[[[  0]
  [ 10]
  [ 20]
  [ 30]]

 [[ 10]
  [ 20]
  [ 30]
  [ 40]]

 [[ 20]
  [ 30]
  [ 40]
  [ 50]]

 [[ 30]
  [ 40]
  [ 50]
  [ 60]]

 [[ 40]
  [ 50]
  [ 60]
  [ 70]]

 [[ 50]
  [ 60]
  [ 70]
  [ 80]]

 [[ 60]
  [ 70]
  [ 80]
  [ 90]]

 [[ 70]
  [ 80]
  [ 90]
  [100]]

 [[ 80]
  [ 90]
  [100]
  [110]]

 [[ 90]
  [100]
  [110]
  [120]]]


array([[ 40.],
       [ 51.],
       [ 60.],
       [ 70.],
       [ 80.],
       [ 90.],
       [100.],
       [110.],
       [120.],
       [130.]], dtype=float32)